In [1]:
#Read in the extracted brown files
import glob

tagged_files = glob.glob("_extracted_brown/*.txt") #Read in the files and creates a list
print(type(tagged_files))
print(len(tagged_files)) #Should be 500

<class 'list'>
500


In [ ]:
'''
Make the words in the files a dicitonary
That way {word, pos_tag}
'''
#I got help from the website where we download the extarcted brown text files
#https://kristopherkyle.github.io/Corpus-Linguistics-Working-Group/pos_tagging_1.html

#divide into sentences
full_data:list = []
for file in tagged_files:
    with open(file, 'r') as x:
        text = x.split("\n\n") #split into sentences
        for sent in text: #iterate through sentences
            sentence = []
            for word_line in sent.split("\n"): #iterate through words
                #Skip empty lines and lines without
                if not word_line.strip() and " " in word_line:
                    continue
                pos_set: set = set()
                word_, pos = word_line.split(" ", 1) #That 1 is the max count (split once)
                pos_set = set(pos.split("|")) #Preserve all possible tags associate with that word
            if sentence:
                full_data.append(sentence)
	

In [ ]:
#Sanity Check to see if this correct
print(type(full_data))


<class 'list'>
1170811


In [ ]:
#HMM Model
import numpy as np
class HiddenMarkovModel:
    def __init__(self, states, observations):
        self.states = states
        self.observations = observations
        self.n_states = len(states)
        self.n_observations= len(observations)
        #Create empty initial/transition/emission probabilities since these are calculated
        #based on the tags and words
        self.initial_probs = np.zeros((self.states))
        self.transition_probs = np.zeros((self.n_states, self.n_states))
        self.emission_probs = np.zeros((self.n_states, self.n_observations))
        
    def transition_probabilities():
        """
        Look at each
        Create the transition probability
        Returns:
            trans_prob: a dicitonary of 
        """
        trans_prob = {}
        return trans_prob
    
    def emission_probabilities():
        """
        Look at 
        Create the emission probability

        Returns:
            emission_pron: a dicitonary 
        """
        emission_prob = {}
        return emission_prob
    
    def viterbi(self):
        """ My implementation of the viterbi algorithm from the textbook"""
        return 
    def generate_sequence():
        sequence = []
        print("Generated sequence: ")
        return sequence
    def prediction():
        return
        

In [ ]:
#Now I need to read in the GMB dataset


In [ ]:
#Feature engineering necessary for the CRF
def word_feature():
    features = 
    return features

In [ ]:
#CRF 
class ConditionalRandomField():
    def __init__(self):
        self.